In [1]:
%pip install hydra-core
%pip install sentencepiece
%pip install transformers

     |████████████████████████████████| 133kB 7.0MB/s 
     |████████████████████████████████| 112kB 10.8MB/s 
     |████████████████████████████████| 645kB 11.3MB/s 
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-cp37-none-any.whl size=141231 sha256=80350743a6988c4b968a006fb99897d892503817dd6549faf03afa5d2b22680d
  Stored in directory: /root/.cache/pip/wheels/e3/e2/fa/b78480b448b8579ddf393bebd3f47ee23aa84c89b6a78285c8
Successfully built antlr4-python3-runtime
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2MB 8.5MB/s 
     |████████████████████████████████| 2.0MB 7.5MB/s 
     |████████████████████████████████| 3.2MB 34.4MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=c42f686c97ce12959a8b3baaaec657214835140ab14dca6a087a4636fd09b312
  S

In [ ]:
import os
import json
import time
import torch
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from torch.utils.data import TensorDataset, random_split, \
                            DataLoader, RandomSampler, SequentialSampler
from transformers import CamembertForSequenceClassification, CamembertTokenizer, \
                         AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
# Functions : preprocess() (create dataloaders from raw data) 
# load_models() (load tokenizers and models) training() (loop of one training step) evaluate()

In [ ]:
    df = pd.read_csv("scraped_reviews.csv", sep=";")


In [ ]:
display(df)

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category
0,Ghestin K,1,5.0,Les meilleurs,"Large catalogue, SAV a l’écoute et compréhensi...",15 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
1,Sylvain,2,5.0,Un large choix de produits,"Un large choix de produits, du Matos de qualit...",15 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
2,Helrat,4,5.0,Toujours parfait,"Toujours parfait. Depuis 4 ans, je commande to...",14 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
3,Laura Perrault,1,5.0,Site aussi bon que d'habitude ou l'on…,Site aussi bon que d'habitude ou l'on trouve p...,8 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
4,HENRY KARINE,2,5.0,Super!,"Présentation agréable, conseils rapides et ada...",11 mars 2021,www.lepetitvapoteur.com,food_beverages_tobacco
...,...,...,...,...,...,...,...,...
141001,Florian,1,5.0,Pneu,"J'ai commandé quatre pneus, ils sont arrivés e...",NaN,www.tonnycat.com,vehicles_transportation
141002,Dorian Bois,2,5.0,Très bon produit simple au montage et…,Très bon produit simple au montage et très con...,NaN,www.tonnycat.com,vehicles_transportation
141003,Alexis De Wilde,1,5.0,Excellent site,"Excellent site, auquel je me suis facilement f...",NaN,www.tonnycat.com,vehicles_transportation
141004,Georges LANCERON,1,5.0,Très bon produit et livraison ultra…,Très bon produit et livraison ultra rapide !Me...,NaN,www.tonnycat.com,vehicles_transportation


In [ ]:
df = df.dropna(subset=['content_review'])
df['stars'].value_counts()

5.0    108835
4.0     11983
3.0      2084
1.0      2070
2.0       946
Name: stars, dtype: int64

In [ ]:
df["sentiment"] = df["stars"].replace({1: 0, 2: 0, 3: 2, 4: 1, 5:1}, inplace=False)
df_neg = df[df['sentiment']==0]
df_pos = df[df['sentiment']==1][:3016]
df_review = pd.concat([df_neg, df_pos], ignore_index=True)
df_review = df_review.sample(frac=1).reset_index(drop=True)
df_review['sentiment'] = df_review['sentiment'].astype(int, copy=False)
df_review.info()
df_review['sentiment'].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6032 entries, 0 to 6031
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   consumer_name              6031 non-null   object 
 1   consumer_nb_review_writed  6032 non-null   int64  
 2   stars                      6032 non-null   float64
 3   title_review               6032 non-null   object 
 4   content_review             6032 non-null   object 
 5   date_experience            1481 non-null   object 
 6   company                    6032 non-null   object 
 7   category                   6032 non-null   object 
 8   sentiment                  6032 non-null   int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 424.2+ KB


1    3016
0    3016
Name: sentiment, dtype: int64

In [ ]:
# train = {"reviews": [], "sentiment": []}
# #for_train = {"reviews": [], "sentences": []}
# count_1 = 0
# count_2 = 0
# count_3 = 0
# count_4 = 0
# count_5 = 0
# idx = -1
# nbr = 1000
# for i in df['stars']:
#     idx = idx + 1
#     if i == 1:
#         index = df.index[idx]
#         count_1 = count_1 + 1
#         if count_1 < nbr:
#             reviews = df['content_review'][index]
#             sentiment = df['stars'][index]
#             train["reviews"].append(reviews)
#             train["sentiment"].append(sentiment)
#     if i == 2:
#         index = df.index[idx]
#         count_2 = count_2 + 1
#         if count_2 < nbr:
#             reviews = df['content_review'][index]
#             sentiment = df['stars'][index]
#             train["reviews"].append(reviews)
#             train["sentiment"].append(sentiment)
#     if i == 3:
#         index = df.index[idx]
#         count_3 = count_3 + 1
#         if count_3 < 2000:
#             reviews = df['content_review'][index]
#             sentiment = df['stars'][index]
#             train["reviews"].append(reviews)
#             train["sentiment"].append(sentiment)
#     if i == 4:
#         index = df.index[idx]
#         count_4 = count_4 + 1
#         if count_4 < nbr:
#             reviews = df['content_review'][index]
#             sentiment = df['stars'][index]
#             train["reviews"].append(reviews)
#             train["sentiment"].append(sentiment)
#     if i == 5:
#         index = df.index[idx]
#         count_5 = count_5 + 1
#         if count_5 < nbr:
#             reviews = df['content_review'][index]
#             sentiment = df['stars'][index]
#             train["reviews"].append(reviews)
#             train["sentiment"].append(sentiment)
# df_train = pd.DataFrame(data=train)


In [ ]:
df_review

,consumer_name,consumer_nb_review_writed,stars,title_review,content_review,date_experience,company,category,sentiment
0,P. GUEGAN,3,5.0,Livraison conforme et rapide,Bonne réactivité de la Ste et colis bien protégé.,NaN,monwhisky.fr,food_beverages_tobacco,1.0
1,briand,2,5.0,Le « taulier »👍🏻à la prochaine,Le « taulier »👍🏻à la prochaine Continuez et bo...,NaN,entrepotitalien.fr,food_beverages_tobacco,1.0
2,Val St,3,1.0,Catastrophique,Catastrophique. Des délais de réponse (dossier...,11 mars 2021,monexpert-renovation-energie.fr,utilities,0.0
3,Mr CHASSAC,1,5.0,Très bien,Très bien article comme prévu merci,NaN,stephconti.fr,food_beverages_tobacco,1.0
4,Lana Silva Cabral,1,2.0,Mon dossier a été hyper mal suivi,"Mon dossier a été hyper mal suivi, de nombreus...",NaN,partners-finances.fr,money_insurance,0.0
...,...,...,...,...,...,...,...,...,...
6027,SCI,1,1.0,Service très cher sans la prestation attendue,Impossible de contacter ce service. Nous ne sa...,4 mars 2021,digidom.pro,legal_services_government,0.0
6028,VAILLANT MARC,1,5.0,Très bonne cave,"Très bonne cave , du choix, des conseils et de...",NaN,www.comptoirdesvignes.fr,food_beverages_tobacco,1.0
6029,'Pascal Vandy,1,1.0,Une façon de faire de l'argent trés…,Une façon de faire de l'argent trés facile qua...,NaN,saisirprudhommes.com,legal_services_government,0.0
6030,Limongi,1,1.0,Je n’ai pas reçu la marchandise,Je n’ai pas reçu la marchandise! Une réclamati...,7 mars 2021,manubricole.com,construction_manufactoring,0.0


A voir

In [ ]:
#camembert = torch.hub.load('pytorch/fairseq', 'camembert')Besoin d'aide ? Découvrez d'autres conseils concernant les recherches sur Google.
#camembert.eval()
TOKENIZER = CamembertTokenizer.from_pretrained(
    'camembert-base',
    do_lower_case=True)

In [ ]:
reviews = df_review['content_review'].values.tolist()
sentiments = df_review['sentiment'].values.tolist()

In [ ]:
def preprocess(raw_reviews, sentiments=None):
    """
    Cette fonction prends de la donnée brute en argument et retourne un 'dataloader' pytorch

    Args
        raw_reviews (array-like) : Une liste de reviews sous forme de 'str'
        
        sentiments : Une liste 'sentiments' (0 = negatif, 1 = positif) de la meme taille que
                     'raw_review'
    
    Returns
        inputs_ids, attention_masks, sentiments(optionel) : Objet  de PyTorch qui contient 
                    les versions tokenisees et encodees des donnees brutes
    """
    
    encoded_batch = TOKENIZER.batch_encode_plus(raw_reviews,
                                                add_special_tokens=True,
                                                return_attention_mask=True,
                                                padding=True,
                                                truncation=True,
                                                return_tensors = 'pt')
    if sentiments:
        sentiments = torch.tensor(sentiments)
        return encoded_batch['input_ids'], encoded_batch['attention_mask'], sentiments
    return encoded_batch['input_ids'], encoded_batch['attention_mask']

In [ ]:
# Split train-validation
split_border = int(len(sentiments)*0.8)
reviews_train, reviews_validation = reviews[:split_border], reviews[split_border:]
sentiments_train, sentiments_validation = sentiments[:split_border], sentiments[split_border:]

In [ ]:
input_ids, attention_mask, sentiments_train = preprocess(reviews_train, sentiments_train)
# Combine the training inputs into a TensorDataset
train_dataset = TensorDataset(
    input_ids,
    attention_mask,
    sentiments_train)

input_ids, attention_mask, sentiments_validation = preprocess(reviews_validation, sentiments_validation)
# Combine the validation inputs into a TensorDataset
validation_dataset = TensorDataset(
    input_ids,
    attention_mask,
    sentiments_validation)

In [ ]:
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders
train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size)

validation_dataloader = DataLoader(
            validation_dataset,
            sampler = SequentialSampler(validation_dataset),
            batch_size = batch_size)

In [ ]:
try:
    state_dict = torch.load("model_nlp_final.pt")
    print("Loading trained model...")
    model = CamembertForSequenceClassification.from_pretrained(
    'camembert-base', num_labels = 2,
    state_dict=state_dict)
    print("Trained model loaded!")
except Exception as e:
    print("Enable to load trained model.")
    print(e)
    model = CamembertForSequenceClassification.from_pretrained(
        'camembert-base',
        num_labels = 2)

Loading trained model...
Trained model loaded!


# Nouvelle section

/content/model_nlp.pt

/content/scraped_reviews.csv

In [ ]:
def predict(reviews, model=model):
    with torch.no_grad():
        model.eval()
        input_ids, attention_mask = preprocess(reviews)
        retour = model(input_ids, attention_mask=attention_mask)
        print(retour)
        return torch.argmax(retour[0], dim=1)

In [ ]:
def evaluate(reviews, sentiments, metric='report'):
    predictions = predict(reviews)
    if metric == 'report':
        return metrics.classification_report(sentiments, predictions, zero_division=0)
    elif metric == 'matrix':
        return metrics.confusion_matrix(sentiments, predictions)

In [ ]:
def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # Learning Rate - Default is 5e-5
                  eps = 1e-8 # Adam Epsilon  - Default is 1e-8.
                )

In [ ]:
df_review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6032 entries, 0 to 6031
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   consumer_name              6031 non-null   object 
 1   consumer_nb_review_writed  6032 non-null   int64  
 2   stars                      6032 non-null   float64
 3   title_review               6032 non-null   object 
 4   content_review             6032 non-null   object 
 5   date_experience            1481 non-null   object 
 6   company                    6032 non-null   object 
 7   category                   6032 non-null   object 
 8   sentiment                  6032 non-null   int64  
dtypes: float64(1), int64(2), object(6)
memory usage: 424.2+ KB


In [ ]:
# Training loop
training_stats = []
                                                                                
# Measure the total training time for the whole run.
total_t0 = time.time()

epochs = 10

# Total number of training steps is [number of batches] x [number of epochs]
# (Note that this is not the same as the number of training samples)
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

device = torch.device("cuda:0")
model.to(device)

# This variable will evaluate the convergence on the training
consecutive_epochs_with_no_improve = 0

# Training
for epoch in range(0, epochs):
    
    print("")
    print(f'########## Epoch {epoch} / {epochs} ##########')
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode
    model.train()

    # For each batch of training data
    #for step, batch in enumerate(train_dataloader):
    for step, batch in enumerate(tqdm(train_dataloader)):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = time.time() - t0
            
            # Report progress
            print(f'  Batch {step}  of  {len(train_dataloader)}    Elapsed: {format_time(elapsed)}.')

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the 'device' using the 'to' method
        #
        # 'batch' contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: skills 
        input_id = batch[0].to(device)
        attention_mask = batch[1].to(device)
        sentiment = batch[2].to(device)

        # Clear any previously calculated gradients before performing a backward pass
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch)
        # the loss (because we provided skills) and the "logits"--the model
        # outputs prior to activation
        loss = model(input_id, 
                             token_type_ids=None, 
                             attention_mask=attention_mask, 
                             labels=sentiment,
                             return_dict=True)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. 'loss' is a Tensor containing a
        # single value; the '.item()' function just returns the Python value 
        # from the tensor
        total_train_loss += loss.loss.item()

        # Perform a backward pass to calculate the gradients
        loss.loss.backward()

        # Clip the norm of the gradients to 1.0
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches
    avg_train_loss = total_train_loss / len(train_dataloader)   

    if epoch > 0:
        if min([stat['Training Loss'] for stat in training_stats]) <= avg_train_loss:
            # i.e. If there is not improvement
            consecutive_epochs_with_no_improve += 1
        else:
            # If there is improvement
            consecutive_epochs_with_no_improve = 0
            print("Model saved!")
            torch.save(model.state_dict(), "model_nlp.pt")
    
    # Measure how long this epoch took
    training_time = time.time() - t0

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch + 1,
            'Training Loss': avg_train_loss,
            'Training Time': training_time,
        }
    )
    if consecutive_epochs_with_no_improve == 2:
        print("Stop training : The loss has not changed since 2 epochs!")
        break

print("Model saved!")
with open('metrics.json', 'w+') as outfile:
    json.dump(training_stats, outfile)
torch.save(model.state_dict(), "model_nlp_final.pt")

  0%|          | 0/604 [00:00<?, ?it/s]


########## Epoch 0 / 10 ##########
Training...


  7%|▋         | 40/604 [00:59<13:57,  1.49s/it]

  Batch 40  of  604    Elapsed: 0:01:00.


 13%|█▎        | 80/604 [01:58<12:53,  1.48s/it]

...

  Batch 560  of  604    Elapsed: 0:13:45.


 99%|█████████▉| 600/604 [14:44<00:05,  1.48s/it]

  Batch 600  of  604    Elapsed: 0:14:44.


100%|██████████| 604/604 [14:49<00:00,  1.47s/it]


Model saved!

  Average training loss: 0.00
  Training epoch took: 890.3548202514648
Model saved!
